In [11]:
import os
from multiprocessing import Pool
import pandas as pd
import numpy as np
from tqdm import tnrange
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss

In [4]:
big_data=pd.read_hdf('raw_data/big.hdf',key='train')
int_fea_num=13
cat_fea_num=26
for c in tnrange(int_fea_num):
    big_data[c].fillna(big_data[c].quantile(),inplace=True)

HBox(children=(IntProgress(value=0, max=13), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [5]:
label=pd.read_hdf('raw_data/label.hdf',key='label')

In [17]:
tst_len=6042135
val_len=6000000
x_tra,y_tra,x_val,y_val,x_tst=big_data[:big_data.shape[0]-val_len-tst_len],label[:big_data.shape[0]-val_len-tst_len][0].ravel(),big_data[big_data.shape[0]-val_len-tst_len:big_data.shape[0]-tst_len],label[-val_len:][0].ravel(),big_data[big_data.shape[0]-tst_len:]
x_tra.shape,y_tra.shape,x_val.shape,y_val.shape,x_tst.shape

((39840617, 39), (39840617,), (6000000, 39), (6000000,), (6042135, 39))

In [18]:
%%time
clf = LGBMClassifier(n_jobs=12)
clf.fit(x_tra,y_tra,categorical_feature=list(range(int_fea_num,int_fea_num+cat_fea_num)))


/home/cq/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


CPU times: user 56min 29s, sys: 1min 27s, total: 57min 57s
Wall time: 7min 48s


In [19]:
log_loss(y_tra,clf.predict_proba(x_tra)),log_loss(y_val,clf.predict_proba(x_val))

(0.4641715663402575, 0.4719983049737419)

In [23]:
log_loss(y_val,clf.predict_proba(x_val)[:,1])

0.4719983049737419

In [20]:
x_tra,y_tra=big_data[:big_data.shape[0]-tst_len],label[:big_data.shape[0]-tst_len][0].ravel()
x_tra.shape,y_tra.shape

((45840617, 39), (45840617,))

In [24]:
clf = LGBMClassifier(n_jobs=12)
clf.fit(x_tra,y_tra,categorical_feature=list(range(int_fea_num,int_fea_num+cat_fea_num)))

/home/cq/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=12, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [25]:
result=pd.read_csv('raw_data/random_submission.csv',dtype={'Id':'str'})
result.Predicted=clf.predict_proba(x_tst)[:,1]
result[['Id','Predicted']].to_csv('output/lgb.csv',index=False)

In [26]:
!tar -czf output/lgb.gz output/lgb.csv